## Feature Reduction
Preform PCA, mutual information, permutation importance, 

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [82]:
df = pd.read_csv('data/preprocessed_normalized_GWP.csv')
X = df.drop(['GWP'], axis=1)
y = df['GWP']
df

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAtom,nHeavyAtom,nH,nB,...,JGT,VE1_D,VE2_D,VE3_D,VR1_D,VR2_D,SRW5,WTPT-3,XLogP,GWP
0,-0.28046,-0.705055,-0.212377,-0.253202,-0.524342,-0.609286,-0.292024,-0.789803,0.180840,-0.103975,...,-0.702354,-0.625459,-0.578342,0.765718,-0.630200,-0.611651,-0.256579,-0.672318,-0.548392,4.440
1,-0.28046,0.537507,-0.143745,-0.702741,-1.037607,-0.609286,-0.869893,-0.789803,-0.703267,-0.103975,...,0.424499,-0.625459,-0.578342,-0.164891,1.525491,2.162446,-0.256579,-0.232254,0.133326,6.400
2,-0.28046,-0.924560,-0.134876,-0.004550,-0.138494,-0.609286,0.141377,-0.511661,0.622894,-0.103975,...,-0.784520,-0.625459,-0.578342,-0.375595,0.806928,0.867867,-0.256579,-0.661854,-0.240537,2.900
3,-0.28046,-0.713210,-0.209979,0.138308,-0.138494,-0.609286,0.141377,-0.511661,0.622894,-0.103975,...,0.354071,1.220320,1.726728,0.677057,-0.630197,-0.611649,-0.256579,-0.671834,-0.299511,4.290
4,-0.28046,-0.088004,-0.286085,0.212871,-0.138494,-0.609286,0.141377,-0.511661,0.622894,-0.103975,...,0.354071,1.220320,1.726728,0.677057,-0.630197,-0.611649,-0.256579,-0.668588,-0.387702,2.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,-0.28046,-0.166584,-0.288515,-0.139987,-0.590644,-0.609286,-0.436491,-0.511661,-0.261214,-0.103975,...,0.354071,1.220320,1.726728,0.677057,-0.630197,-0.611649,-0.256579,-0.149031,-0.570575,2.870
183,-0.28046,-1.087359,-0.060014,-0.242612,-0.689826,-0.609286,-0.436491,-0.789803,-0.040187,-0.103975,...,-0.702354,-0.625459,-0.578342,0.765718,-0.630200,-0.611651,-0.256579,-0.205316,-1.390802,9.530
184,-0.28046,-0.585166,-0.243344,0.402741,-0.084462,-0.609286,0.141377,-0.233520,0.401867,-0.103975,...,-2.110920,-0.625459,-0.578342,-0.559072,-0.630190,-0.611645,-0.256579,0.048062,-1.012610,15.500
185,-0.28046,3.754845,4.041201,2.626455,4.016982,-0.609286,3.319652,1.713470,3.717271,-0.103975,...,2.935936,-0.625459,-0.578342,-2.315828,2.244076,0.526443,-0.256579,1.964786,1.675308,0.828


## PCA
Retain principle components so that 95% of the oringial variance of the dataset is retained

In [65]:
from sklearn.decomposition import PCA

pca = PCA()
pca_X = pca.fit_transform(X)

k = 1
x = pca.explained_variance_ratio_[0:k].cumsum()
while x < 0.95:
    k+=1
    x = sum(pca.explained_variance_ratio_[0:k])

pca_values = pd.DataFrame(pca_X[:k]).transpose()


In [66]:
pca_values['GWP'] = y
# pca_values.to_csv('data/PCA_components_GWP.csv')

## Mutual Information
Colculate the mutual information score between each variable and GWP. Keep only colums with scores less than 0.01.

In [92]:
from sklearn.feature_selection import mutual_info_regression
columns = df.columns
MI_scores = mutual_info_regression(df.drop(['GWP'], axis = 1), y)
mi_df = df.copy()

for i in range(len(MI_scores)):
    if MI_scores[i] > 0.01:
        mi_df =mi_df.drop(columns[i], axis=1)

mi_df

,nAcid,nAtom,nH,nB,nN,nO,nS,nF,nBr,nI,...,JGI6,JGI7,JGI8,JGI9,VE3_D,VR1_D,VR2_D,SRW5,WTPT-3,GWP
0,-0.28046,-0.292024,0.180840,-0.103975,-0.448094,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,0.765718,-0.630200,-0.611651,-0.256579,-0.672318,4.440
1,-0.28046,-0.869893,-0.703267,-0.103975,-0.448094,-0.880242,-0.165748,1.605307,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,-0.164891,1.525491,2.162446,-0.256579,-0.232254,6.400
2,-0.28046,0.141377,0.622894,-0.103975,-0.448094,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,-0.375595,0.806928,0.867867,-0.256579,-0.661854,2.900
3,-0.28046,0.141377,0.622894,-0.103975,-0.448094,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,0.677057,-0.630197,-0.611649,-0.256579,-0.671834,4.290
4,-0.28046,0.141377,0.622894,-0.103975,-0.448094,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,0.677057,-0.630197,-0.611649,-0.256579,-0.668588,2.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,-0.28046,-0.436491,-0.261214,-0.103975,-0.448094,0.532679,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,0.677057,-0.630197,-0.611649,-0.256579,-0.149031,2.870
183,-0.28046,-0.436491,-0.040187,-0.103975,0.925570,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,0.765718,-0.630200,-0.611651,-0.256579,-0.205316,9.530
184,-0.28046,0.141377,0.401867,-0.103975,0.925570,-0.173782,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,-0.559072,-0.630190,-0.611645,-0.256579,0.048062,15.500
185,-0.28046,3.319652,3.717271,-0.103975,-0.448094,0.532679,-0.165748,-0.170979,-0.093176,-0.073324,...,-0.263846,-0.143626,-0.139635,-0.10934,-2.315828,2.244076,0.526443,-0.256579,1.964786,0.828


In [93]:
# mi_df.to_csv('data/MI_variables.csv')